In [ ]:
from google.colab import files
uploaded = files.upload()

Saving valid_smiles_fingerprint_Avalon.parquet to valid_smiles_fingerprint_Avalon.parquet


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Đọc file Parquet
df = pd.read_parquet('valid_smiles_fingerprint_Avalon.parquet')

In [ ]:
df

,SMILES_Standardized,Activity,Last_Modified_Date,bit_0,bit_1,bit_2,bit_3,bit_4,bit_5,bit_6,...,bit_2038,bit_2039,bit_2040,bit_2041,bit_2042,bit_2043,bit_2044,bit_2045,bit_2046,bit_2047
0,O=[N+]([O-])c1ccc(Cl)c([N+](=O)[O-])c1,Inactive,20120522,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,O=C(O)c1cccc(O)c1O,Inactive,20120522,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,OCC1OC(O)CC(O)C1O,Inactive,20120522,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,O=C(O)CCC(=O)C(=O)O,Inactive,20120522,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,O=C(O)c1ccc(O)c(O)c1,Inactive,20120522,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364301,CC1NNC2C(F)CC(C(=O)Nc3ccc4c(c3)nc(Cc3cc(F)c(-c...,Active,20240109,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,1,1
364302,CCn1cncc1Cn1c(Cc2cc(F)c(-c3cccc(OCc4ccc(Cl)cc4...,Active,20240109,0,0,0,0,1,0,0,...,0,0,1,0,0,1,1,0,1,1
364303,CCn1cncc1Cn1c(Cc2c(F)cc(-c3cccc(OCc4ccc(Cl)cc4...,Active,20240109,0,0,0,0,1,0,0,...,0,0,1,0,0,1,1,0,1,1
364304,CC1NNC2C(F)CC(C(=O)Nc3ccc4c(c3)nc(Cc3cc(F)c(-c...,Active,20240109,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,1,1


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, balanced_accuracy_score,
    precision_recall_curve
)

# ====== 1. Load dữ liệu ======
df = pd.read_parquet("valid_smiles_fingerprint_Avalon.parquet")
df["Activity"] = df["Activity"].map({"Active": 1, "Inactive": 0})
df["Last_Modified_Date"] = pd.to_datetime(df["Last_Modified_Date"], format="%Y%m%d", errors="coerce")

# ====== 2. Temporal split ======
cutoff_date = pd.Timestamp("2022-08-30")
modeling_df = df[df["Last_Modified_Date"] <= cutoff_date].copy()
external_df = df[df["Last_Modified_Date"] > cutoff_date].copy()

print(f"Modeling set: {modeling_df.shape[0]} samples")
print(f"External test set: {external_df.shape[0]} samples")

# ====== 3. Train/Val/Test ======
X = modeling_df.filter(like="bit_")
y = modeling_df["Activity"]

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# ====== Random Forest Model ======
model = RandomForestClassifier(
    n_estimators=300,
    class_weight="balanced_subsample",
    random_state=42,
    n_jobs=-1
)

def find_best_threshold(y_true, y_prob, min_precision=0.9):
    p, r, t = precision_recall_curve(y_true, y_prob)
    p, r, t = p[:-1], r[:-1], t[:-1]
    valid_idx = np.where(p >= min_precision)[0]
    if len(valid_idx) > 0:
        best_idx = valid_idx[np.argmax(r[valid_idx])]
        return t[best_idx], p[best_idx], r[best_idx]
    return 0.5, 0, 0

def evaluate_metrics(y_true, y_prob, threshold):
    y_pred = (y_prob >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "Specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "NPV": tn / (tn + fn) if (tn + fn) > 0 else 0,
        "F1 Score": f1_score(y_true, y_pred, zero_division=0),
        "ROC AUC": roc_auc_score(y_true, y_prob),
    }

# ====== Train ======
print("\n🔍 Training Random Forest...")
model.fit(X_train, y_train)

# ====== Threshold tuning ======
y_val_prob = model.predict_proba(X_val)[:, 1]
best_t, p, r = find_best_threshold(y_val, y_val_prob, min_precision=0.9)
print(f" → Best threshold = {best_t:.3f} (Precision={p:.3f}, Recall={r:.3f})")

# ====== Evaluate ======
y_test_prob = model.predict_proba(X_test)[:, 1]
metrics = evaluate_metrics(y_test, y_test_prob, best_t)
results = pd.DataFrame([metrics], index=["RandomForest"]).round(4)

print("\n📊 Final Test Results:")
print(results)

results.to_csv("results_randomforest.csv")
joblib.dump({"RandomForest": best_t}, "best_threshold_RF.joblib")
joblib.dump(model, "RandomForest.joblib")

print("\n✅ Done — Random Forest saved")


Modeling set: 363525 samples
External test set: 781 samples
Train: (290820, 2048), Val: (36352, 2048), Test: (36353, 2048)

🔍 Training Random Forest...
 → Best threshold = 0.113 (Precision=0.909, Recall=0.976)

📊 Final Test Results:
              Accuracy  Balanced Accuracy  Precision  Recall  Specificity  \
RandomForest    0.9999             0.9756      0.975  0.9512          1.0   

                 NPV  F1 Score  ROC AUC  
RandomForest  0.9999     0.963   0.9877  

✅ Done — Random Forest saved


In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, balanced_accuracy_score
)

# ====== Load model & threshold ======
model = joblib.load("RandomForest.joblib")
best_t = joblib.load("best_threshold_RF.joblib")["RandomForest"]

print(f"Loaded model. Best threshold = {best_t:.3f}")

# ====== Load external dataset ======
df = pd.read_parquet("valid_smiles_fingerprint_Avalon.parquet")
df["Activity"] = df["Activity"].map({"Active": 1, "Inactive": 0})
df["Last_Modified_Date"] = pd.to_datetime(df["Last_Modified_Date"], format="%Y%m%d", errors="coerce")

cutoff_date = pd.Timestamp("2022-08-30")
external_df = df[df["Last_Modified_Date"] > cutoff_date].copy()

X_ext = external_df.filter(like="bit_")
y_ext = external_df["Activity"]

print(f"External test samples: {len(X_ext)}")

# ====== Evaluate function ======
def evaluate_metrics(y_true, y_prob, threshold):
    y_pred = (y_prob >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()

    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "Specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "NPV": tn / (tn + fn) if (tn + fn) > 0 else 0,
        "F1 Score": f1_score(y_true, y_pred, zero_division=0),
        "ROC AUC": roc_auc_score(y_true, y_prob),
    }

# ====== Predict on external ======
y_ext_prob = model.predict_proba(X_ext)[:, 1]

# ====== Evaluate ======
metrics_ext = evaluate_metrics(y_ext, y_ext_prob, best_t)
results = pd.DataFrame([metrics_ext], index=["External_RF"]).round(4)

print("\n📦 External Test Results:")
print(results)

# ====== Save external results ======
results.to_csv("external_results_RF.csv")
print("\n✅ Saved as external_results_RF.csv")


Loaded model. Best threshold = 0.113
External test samples: 781

📦 External Test Results:
             Accuracy  Balanced Accuracy  Precision  Recall  Specificity  \
External_RF    0.5941             0.5316     0.9678  0.5989       0.4643   

                NPV  F1 Score  ROC AUC  
External_RF  0.0413      0.74   0.5735  

✅ Saved as external_results_RF.csv


In [ ]:
from google.colab import files
files.download("external_results_RF.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, balanced_accuracy_score
)

# ====== 1. Load dữ liệu ======
df = pd.read_parquet("valid_smiles_fingerprint_Avalon.parquet")
df["Activity"] = df["Activity"].map({"Active": 1, "Inactive": 0})
df["Last_Modified_Date"] = pd.to_datetime(df["Last_Modified_Date"], format="%Y%m%d", errors="coerce")

# ====== 2. Temporal split ======
cutoff_date = pd.Timestamp("2022-08-30")
modeling_df = df[df["Last_Modified_Date"] <= cutoff_date].copy()
external_df = df[df["Last_Modified_Date"] > cutoff_date].copy()

print(f"Modeling set: {modeling_df.shape[0]} samples")
print(f"External test set: {external_df.shape[0]} samples")

# ====== 3. Train/Val/Test ======
X = modeling_df.filter(like="bit_")
y = modeling_df["Activity"]

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# ====== Train Random Forest ======
print("\n🔍 Training Random Forest...")
model = RandomForestClassifier(
    n_estimators=300,
    class_weight="balanced_subsample",
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

# ====== Evaluate with default threshold = 0.5 ======
threshold = 0.5

def evaluate_metrics(y_true, y_prob, threshold=0.5):
    y_pred = (y_prob >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "Specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "NPV": tn / (tn + fn) if (tn + fn) > 0 else 0,
        "F1 Score": f1_score(y_true, y_pred, zero_division=0),
        "ROC AUC": roc_auc_score(y_true, y_prob)
    }

y_test_prob = model.predict_proba(X_test)[:, 1]
metrics = evaluate_metrics(y_test, y_test_prob, threshold)
results = pd.DataFrame([metrics], index=["RandomForest"]).round(4)

print("\n📊 Final Test Results (Threshold = 0.5):")
print(results)

# Save
results.to_csv("results_randomforest_default05.csv")
joblib.dump({"RandomForest": threshold}, "best_threshold_RF.joblib")
joblib.dump(model, "RandomForest.joblib")

print("\n✅ Done — Random Forest saved with threshold = 0.5")


FileNotFoundError: [Errno 2] No such file or directory: 'valid_smiles_fingerprint_Avalon.parquet'